In [6]:
from google.colab import files
uploaded = files.upload()

In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier as RandomForest
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
all = pd.concat([train, test], axis =0).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [8]:
def kesson_table(df):
    null_val = df.isnull().sum()
    percent = 100 * df.isnull().sum()/len(df)
    kesson_table = pd.concat([null_val, percent], axis=1)
    kesson_table_ren_columns = kesson_table.rename(
        columns = {0 : '欠損数', 1 : '%'})
    return kesson_table_ren_columns

kesson_table(all)

,欠損数,%
Age,263,20.091673
Cabin,1014,77.463713
Embarked,2,0.152788
Fare,1,0.076394
Name,0,0.000000
Parch,0,0.000000
PassengerId,0,0.000000
Pclass,0,0.000000
Sex,0,0.000000
SibSp,0,0.000000


In [0]:
all["surname"] = all["Name"].apply(lambda x:x.split(",")[0])

In [10]:
all[1040: 1050]

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,surname
1040,30.0,NaN,S,26.0000,"Lahtinen, Rev. William",1,1041,2,male,1,NaN,250651,Lahtinen
1041,23.0,C54,C,83.1583,"Earnshaw, Mrs. Boulton (Olive Potter)",1,1042,1,female,0,NaN,11767,Earnshaw
1042,NaN,NaN,C,7.8958,"Matinoff, Mr. Nicola",0,1043,3,male,0,NaN,349255,Matinoff
1043,60.5,NaN,S,NaN,"Storey, Mr. Thomas",0,1044,3,male,0,NaN,3701,Storey
1044,36.0,NaN,S,12.1833,"Klasen, Mrs. (Hulda Kristina Eugenia Lofqvist)",2,1045,3,female,0,NaN,350405,Klasen
1045,13.0,NaN,S,31.3875,"Asplund, Master. Filip Oscar",2,1046,3,male,4,NaN,347077,Asplund
1046,24.0,NaN,S,7.5500,"Duquemin, Mr. Joseph",0,1047,3,male,0,NaN,S.O./P.P. 752,Duquemin
1047,29.0,C97,S,221.7792,"Bird, Miss. Ellen",0,1048,1,female,0,NaN,PC 17483,Bird
1048,23.0,NaN,S,7.8542,"Lundin, Miss. Olga Elida",0,1049,3,female,0,NaN,347469,Lundin
1049,42.0,D22,S,26.5500,"Borebank, Mr. John James",0,1050,1,male,0,NaN,110489,Borebank


In [11]:
all['Age'].fillna(all['Age'].mode()[0], inplace=True)
all["Embarked"] = all["Embarked"].fillna("S")
all.Fare[1043] = all.Fare.mean()

kesson_table(all)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,欠損数,%
Age,0,0.000000
Cabin,1014,77.463713
Embarked,0,0.000000
Fare,0,0.000000
Name,0,0.000000
Parch,0,0.000000
PassengerId,0,0.000000
Pclass,0,0.000000
Sex,0,0.000000
SibSp,0,0.000000


In [0]:
cols = ["surname", "Sex","Embarked"]
all_to_cat = pd.get_dummies(all, columns=cols)

In [13]:
data_counts = all['surname'].value_counts()
surname_to_drop = data_counts[data_counts < 4].index.tolist()

to_drop_list = []
target_col = "surname"

for each_data in surname_to_drop:
    col_name = "{}_{}".format(target_col, each_data)
    to_drop_list.append(col_name)
    
all_to_cat.drop(columns=to_drop_list, inplace=True)

to_drop = ["PassengerId", "Name", "Ticket", "Cabin"]

all_to_cat.drop(columns=to_drop, inplace=True)
print(all_to_cat.shape)

(1309, 53)


In [0]:
train_to_cat = all_to_cat.iloc[0:891, :]
train_train = train_to_cat.drop(["Survived"], axis=1)

In [0]:
target = train_to_cat["Survived"].values
features = train_train.values

In [0]:
x_train, x_test, t_train, t_test = train_test_split(features, target, test_size =0.3, random_state = 0)

In [0]:
model = RandomForest(n_estimators=100).fit(x_train, t_train)

In [18]:
model.score(x_test, t_test)

0.832089552238806

In [19]:
header = np.array(train.drop(['Survived', 'PassengerId'], axis=1).columns)
print(header)
print(model.feature_importances_)

['Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare' 'Cabin'
 'Embarked']
[2.42429437e-01 2.22774776e-01 3.50871131e-02 7.76480217e-02
 4.28190624e-02 5.93473122e-03 6.75409731e-03 3.68987512e-03
 2.06744356e-03 1.55059730e-03 2.29653344e-04 1.70732408e-03
 1.66047612e-03 4.67610925e-03 6.64068904e-04 1.44708289e-04
 8.72724426e-04 2.80162990e-03 9.12065156e-04 3.39167644e-03
 1.38452036e-03 3.86395830e-04 1.49334229e-04 2.62206481e-03
 2.49421438e-04 7.47253477e-04 3.68153568e-04 6.38708959e-04
 2.88318411e-04 1.51488105e-03 1.36978424e-03 6.81685110e-04
 7.21675921e-04 2.19721440e-03 2.67578686e-04 1.87258683e-03
 6.60710296e-04 1.38848842e-03 0.00000000e+00 2.40703442e-03
 8.98026335e-04 1.45600994e-03 0.00000000e+00 1.54257012e-03
 0.00000000e+00 8.01634552e-04 4.62900670e-03 1.45357660e-01
 1.31155094e-01 1.33713562e-02 9.39453864e-03 1.36627030e-02]


In [0]:
test_to_cat = all_to_cat.iloc[891:, :]
test_test = test_to_cat.drop(["Survived"], axis=1)

In [0]:
test_features = test_test.values

In [22]:
preds = model.predict(test_features)
preds

array([0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 1., 1., 1.

欠損があると出来ない。埋めよう。→埋めたら出来た

In [0]:
PassengerId = np.array(test["PassengerId"]).astype(int)
                            
my_solution = pd.DataFrame(preds, PassengerId, columns = ["Survived"])
                            
my_solution.to_csv("GRY.csv", index_label = ["PassengerId"])

In [0]:
from google.colab import files
files.download("GRY.csv")

0になってしまった。何故だろう、、仕方がないので自分で評価してみる。

In [25]:
uploaded = files.upload()

In [26]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

GS = pd.read_csv("gender_submission.csv")
GS

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [0]:
Correction = GS["Survived"].values

In [28]:
print('Accuracy score = \t {}'.format(accuracy_score(Correction, preds)))
print('Precision score = \t {}'.format(precision_score(Correction, preds)))
print('Recall score =   \t {}'.format(recall_score(Correction, preds)))
print('F1 score =      \t {}'.format(f1_score(Correction, preds)))

Accuracy score = 	 0.8373205741626795
Precision score = 	 0.7692307692307693
Recall score =   	 0.7894736842105263
F1 score =      	 0.7792207792207793


In [30]:
preds.astype('int32')

array([0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [0]:
PassengerId = np.array(test["PassengerId"]).astype(int)
                            
my_solution = pd.DataFrame(preds, PassengerId, columns = ["Survived"])
                            
my_solution.to_csv("GRINT.csv", index_label = ["PassengerId"])

In [0]:
from google.colab import files
files.download("GRINT.csv")

0になる。何故だ→コピーして提出したら76%。